In [ ]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [ ]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
!{sys.executable} -m pip install torchvision

In [ ]:
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

In [ ]:
with open('../data/train_opt.pkl', mode='rb') as f:
    opt = pickle.load(f)
    
iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
    
start_epoch, epoch_iter = 1, 0
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

In [ ]:
model = create_model(opt)
visualizer = Visualizer(opt)

In [ ]:
for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
    for i, data in enumerate(dataset, start=epoch_iter):
        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        save_fake = total_steps % opt.display_freq == display_delta
        
        ############## Forward Pass ######################
        losses, generated = model(Variable(data['label']), Variable(data['inst']), 
            Variable(data['image']), Variable(data['feat']), infer=save_fake)
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
        loss_G = loss_dict['G_GAN'] + loss_dict.get('G_GAN_Feat',0) + loss_dict.get('G_VGG',0)
        
        ############### Backward Pass ####################
        # update generator weights
        model.module.optimizer_G.zero_grad()
        loss_G.backward()
        model.module.optimizer_G.step()

        # update discriminator weights
        model.module.optimizer_D.zero_grad()
        loss_D.backward()
        model.module.optimizer_D.step()
        
        #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]) 

        ############## Display results and errors ##########
        ### print out errors
        if total_steps % opt.print_freq == print_delta:
            errors = {k: v.data[0] if not isinstance(v, int) else v for k, v in loss_dict.items()}
            t = (time.time() - iter_start_time) / opt.batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            visualizer.plot_current_errors(errors, total_steps)

        ### display output images
        if save_fake:
            visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0], opt.label_nc)),
                                   ('synthesized_image', util.tensor2im(generated.data[0])),
                                   ('real_image', util.tensor2im(data['image'][0]))])
            visualizer.display_current_results(visuals, epoch, total_steps)

        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.module.save('latest')            
            np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

        if epoch_iter >= dataset_size:
            break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()
        
torch.cuda.empty_cache()

In [ ]:
opt.batchSize

In [ ]:
d ={2, 8, 10, 11, 2060, 4107, 4110, 17, 4115, 2068, 4118, 4119, 26, 2074, 4122, 4124, 4130, 36, 2085, 40, 2088, 4139, 4140, 46, 2095, 49, 60, 67, 2120, 2128, 82, 4178, 93, 2150, 4198, 2158, 4207, 112, 113, 4211, 118, 4214, 2169, 2173, 128, 137, 2186, 2187, 141, 2189, 145, 2194, 4242, 2196, 4249, 156, 2213, 166, 4261, 4264, 170, 4268, 2225, 2227, 2229, 182, 183, 187, 2236, 4283, 4286, 2239, 4293, 201, 2249, 2251, 4303, 2256, 210, 2258, 2261, 214, 2263, 4309, 4311, 2266, 4313, 220, 4314, 2270, 4317, 226, 2276, 2277, 2279, 4328, 2281, 2288, 2292, 247, 2296, 250, 4347, 254, 2303, 4352, 257, 4355, 2315, 4364, 272, 2320, 2321, 4373, 2326, 4375, 2333, 287, 289, 2350, 4399, 2352, 4400, 4405, 2364, 4414, 4418, 4421, 2376, 329, 2379, 2381, 4431, 340, 2388, 342, 343, 345, 346, 347, 348, 349, 2393, 4442, 4445, 4450, 356, 2405, 2407, 361, 368, 4468, 4472, 4473, 2426, 2427, 4477, 382, 2433, 2436, 4486, 391, 2440, 4488, 4494, 2450, 2452, 405, 2454, 4501, 2456, 2459, 413, 2465, 2466, 2467, 4515, 2471, 4519, 4522, 4523, 429, 430, 2477, 433, 2481, 437, 4535, 440, 441, 2490, 4542, 4544, 450, 2498, 453, 4551, 456, 2504, 2507, 463, 4559, 467, 4566, 476, 2525, 4573, 481, 483, 484, 488, 4586, 2540, 2543, 496, 4591, 2546, 501, 2552, 506, 4604, 4611, 4612, 2565, 4616, 521, 2569, 524, 4622, 527, 2575, 4630, 2583, 2589, 2590, 2592, 545, 547, 559, 567, 4665, 2618, 576, 4672, 578, 579, 580, 2627, 4674, 2631, 4677, 586, 587, 2636, 4685, 590, 591, 4688, 594, 2645, 598, 4693, 4695, 4705, 2662, 618, 4714, 2672, 629, 630, 4730, 635, 636, 2683, 638, 4736, 4742, 649, 652, 2703, 2704, 2707, 4757, 667, 4764, 4765, 2719, 2723, 4771, 4772, 679, 681, 2731, 2734, 4783, 2737, 692, 694, 2742, 697, 705, 2753, 4802, 4806, 2763, 2766, 2767, 2768, 721, 724, 725, 2773, 4822, 4828, 2781, 735, 736, 2783, 738, 4831, 4834, 2790, 2793, 4843, 750, 751, 752, 753, 2798, 4852, 2807, 2813, 766, 768, 769, 4866, 771, 772, 773, 774, 775, 4870, 777, 2826, 4873, 4876, 4884, 4891, 798, 4894, 802, 2853, 2854, 4904, 2859, 815, 4911, 819, 820, 821, 822, 4915, 825, 2876, 835, 2884, 840, 4936, 842, 2891, 4937, 846, 848, 852, 4949, 2902, 4954, 4955, 4958, 863, 867, 2917, 873, 4970, 4971, 2924, 2930, 883, 4981, 2934, 889, 890, 891, 2938, 894, 2942, 2948, 2949, 2952, 2954, 2958, 911, 912, 2959, 5007, 916, 2971, 5020, 2973, 929, 5026, 933, 2982, 2983, 5029, 5030, 5032, 2995, 952, 5049, 955, 963, 3015, 5063, 5069, 5070, 979, 980, 987, 5083, 3045, 5095, 1000, 1001, 3053, 1007, 1009, 1010, 3058, 3061, 5109, 5112, 5113, 5115, 1021, 1022, 3072, 1025, 3076, 3077, 3079, 5127, 3084, 3086, 1041, 3089, 1043, 5137, 5141, 5144, 3098, 1060, 3109, 5156, 1063, 5159, 3113, 5161, 5162, 3118, 3120, 1073, 1074, 3122, 5170, 1077, 5179, 1084, 3132, 3136, 5184, 3143, 3147, 5198, 1103, 1105, 5202, 3156, 5204, 3158, 5207, 3160, 5210, 3163, 5214, 3169, 1123, 3175, 1128, 1131, 1132, 3179, 1136, 5233, 1139, 1153, 3202, 3204, 1157, 3208, 1163, 5260, 1167, 3216, 5264, 3218, 3219, 1172, 3221, 5265, 5271, 3227, 3228, 3229, 5287, 5288, 5295, 1200, 5297, 3251, 1205, 3254, 5303, 5305, 3259, 5307, 5308, 1215, 1216, 1221, 1222, 3273, 5323, 5326, 3279, 5327, 5328, 5332, 1242, 5342, 1251, 1254, 3302, 3304, 1258, 5355, 1263, 5361, 1267, 1268, 1272, 3320, 3321, 5370, 5371, 5374, 3328, 1282, 1287, 5389, 3344, 3348, 3350, 1304, 3353, 5404, 1310, 3360, 5411, 1316, 5415, 1320, 5426, 1331, 1332, 1336, 1338, 5434, 1341, 3389, 1346, 5443, 1349, 1352, 1353, 3400, 3402, 5451, 5452, 3406, 1359, 3408, 5463, 1368, 1369, 1370, 3417, 3420, 5464, 5479, 1384, 5480, 1390, 3439, 3441, 3443, 5491, 1401, 1404, 3454, 1408, 3457, 1410, 5504, 3461, 3466, 5516, 3469, 3471, 1426, 1428, 1429, 5526, 5527, 3480, 1433, 5536, 3490, 3492, 5541, 1446, 1448, 5545, 1452, 3500, 3502, 5553, 3508, 3512, 5567, 1474, 5578, 1483, 3533, 3534, 3538, 1492, 1493, 1494, 3540, 1496, 5593, 5596, 3550, 1504, 3552, 3553, 5602, 5603, 1509, 3558, 5608, 1513, 5611, 5612, 1518, 1519, 5617, 1522, 1524, 5620, 5622, 1531, 1532, 1537, 1542, 1545, 3595, 3597, 5646, 3599, 5647, 5650, 1556, 3606, 3611, 5663, 3617, 5665, 3620, 5671, 1579, 1580, 5675, 3632, 5680, 5681, 1595, 5691, 5692, 3649, 5699, 1607, 3656, 3657, 5703, 1612, 5709, 5710, 3668, 3673, 1631, 5727, 1640, 5736, 1643, 3691, 5741, 3695, 1648, 5745, 5749, 1654, 3702, 3703, 1660, 1663, 3711, 1667, 1668, 5765, 5767, 3721, 1678, 5775, 1681, 5778, 5781, 1686, 1691, 3739, 3740, 5789, 3744, 3746, 5794, 5798, 3753, 1706, 3754, 3755, 3756, 1710, 3757, 3758, 5815, 5816, 1722, 1725, 5826, 3779, 3780, 5827, 3782, 5831, 5832, 3787, 1741, 5837, 1752, 1754, 3802, 3803, 5850, 1758, 3807, 5852, 5857, 3812, 1765, 3817, 1771, 3824, 5873, 3826, 1779, 5879, 5883, 3836, 5884, 3838, 5886, 1792, 3843, 1796, 3844, 5902, 5905, 5907, 3860, 5913, 5914, 1820, 1821, 3871, 3872, 3873, 5919, 1827, 5922, 1829, 1830, 3877, 3879, 1839, 1842, 3891, 1844, 3892, 3894, 1848, 1849, 3897, 5944, 5949, 5950, 5951, 1857, 3907, 5956, 5957, 3910, 3912, 1867, 3917, 5969, 1875, 3924, 3928, 5976, 3934, 5984, 1889, 5988, 5990, 1899, 3947, 1901, 1903, 1907, 1908, 1909, 3958, 1919, 3970, 1926, 3985, 3993, 3995, 3999, 4002, 4003, 1962, 1964, 1966, 4016, 4017, 1970, 1971, 1973, 4022, 1975, 1976, 4025, 4026, 4029, 4032, 4035, 4040, 1995, 4044, 2001, 4050, 2003, 2004, 4057, 2010, 2011, 4061, 2015, 4064, 2017, 4065, 4067, 2020, 2022, 4070, 2028, 2029, 4076, 4078, 2039, 4088, 2043, 2045, 4095}
len(d)

In [ ]:
from shutil import copyfile
import os 


img_path = r"../data/target/train/train_img"

for n in d:
    name = f'img_{n:05d}.png'
    path = os.path.join(img_path,name)
    print(name)
    new_path = os.path.join("../data/target/train/upload", name)
    copyfile(path,new_path )
